In [ ]:
!pip install map_boxes
!pip install tqdm
!pip install pycocotools

In [2]:
from map_boxes import mean_average_precision_for_boxes
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from pycocotools.coco import COCO

In [3]:
GT_JSON = '/opt/ml/detection/dataset/train.json'
PRED_CSV = 'submission_effcient_det_d3_50_train_batch_size_4.csv'


    
# load ground truth
with open(GT_JSON, 'r') as outfile:
    test_anno = (json.load(outfile))

# load prediction
pred_df = pd.read_csv(PRED_CSV)

   

In [4]:
'''
[
    [file_name 1, confidence_score, x_min, x_max, y_min, y_max], 
    [file_name 2 confidence_score, x_min, x_max, y_min, y_max],
    ,,,
    [file_name , confidence_score, x_min, x_max, y_min, y_max]
]
'''
    
new_pred = []

file_names = pred_df['image_id'].values.tolist()
bboxes = pred_df['PredictionString'].values.tolist()
    
'''
create new_pred
'''
    
for i, bbox in enumerate(bboxes):
    if isinstance(bbox, float):
        print(f'{file_names[i]} empty box')

for file_name, bbox in tqdm(zip(file_names, bboxes)):
    boxes = np.array(str(bbox).split(' '))
    
    if len(boxes) % 6 == 1:
        boxes = boxes[:-1].reshape(-1, 6)
    elif len(boxes) % 6 == 0:
        boxes = boxes.reshape(-1, 6)
    else:
        raise Exception('error', 'invalid box count')
    for box in boxes:
        new_pred.append([file_name, box[0], box[1], float(box[2]), float(box[4]), float(box[3]), float(box[5])])

train/0000.jpg empty box
train/0010.jpg empty box
train/0011.jpg empty box
train/0017.jpg empty box
train/0024.jpg empty box
train/0025.jpg empty box
train/0040.jpg empty box
train/0047.jpg empty box
train/0084.jpg empty box
train/0099.jpg empty box
train/0111.jpg empty box
train/0153.jpg empty box
train/0191.jpg empty box
train/0274.jpg empty box
train/0278.jpg empty box
train/0283.jpg empty box
train/0301.jpg empty box
train/0318.jpg empty box
train/0341.jpg empty box
train/0363.jpg empty box
train/0406.jpg empty box
train/0410.jpg empty box
train/0438.jpg empty box
train/0447.jpg empty box
train/0466.jpg empty box
train/0491.jpg empty box
train/0496.jpg empty box
train/0512.jpg empty box
train/0551.jpg empty box
train/0644.jpg empty box
train/0705.jpg empty box
train/0709.jpg empty box
train/0713.jpg empty box
train/0719.jpg empty box
train/0723.jpg empty box
train/0724.jpg empty box
train/0752.jpg empty box
train/0770.jpg empty box
train/0771.jpg empty box
train/0817.jpg empty box


4883it [00:00, 15598.79it/s]


In [5]:
'''
[
    [file_name 1, confidence_score, x_min, x_max, y_min, y_max], 
    [file_name 2, confidence_score, x_min, x_max, y_min, y_max],
    ,,,
    [file_name , confidence_score, x_min, x_max, y_min, y_max]
]
'''
    
gt = []

'''
create gt
'''
    
coco = COCO(GT_JSON)
   
'''
coco.getImgIds(): return image id list
    
coco.loadImgs(image_id): return image_info
    
image_info['file_name']: return file name
   
coco.getAnnIds(imgIds=image_info['id']): return annotation id
    
coco.loadAnns(ann_ids): return annotation information list (annotation_info_list)
    
annotation_info_list[i]['bbox']: return i'th annotation [x_min, y_min, w, h]
    
annotation_info_list[i]['category_id']: return i'th annotation category
    
'''
    
for image_id in coco.getImgIds():
        
    image_info = coco.loadImgs(image_id)[0]
    annotation_id = coco.getAnnIds(imgIds=image_info['id'])
    annotation_info_list = coco.loadAnns(annotation_id)
        
    file_name = image_info['file_name']
        
    for annotation in annotation_info_list:
        gt.append([file_name, annotation['category_id'],
                   float(annotation['bbox'][0]),
                   float(annotation['bbox'][0]) + float(annotation['bbox'][2]),
                   float(annotation['bbox'][1]),
                   (float(annotation['bbox'][1]) + float(annotation['bbox'][3]))])

loading annotations into memory...
Done (t=0.22s)
creating index...
index created!


In [7]:
print(gt[:10])

[['train/0000.jpg', 0, 197.6, 745.4, 193.7, 663.4], ['train/0001.jpg', 3, 0.0, 57.6, 407.4, 588.0], ['train/0001.jpg', 7, 0.0, 144.6, 455.6, 637.2], ['train/0001.jpg', 4, 722.3, 996.5999999999999, 313.4, 565.3], ['train/0001.jpg', 5, 353.2, 586.9, 671.0, 774.4], ['train/0001.jpg', 5, 3.7, 781.9000000000001, 448.5, 690.5], ['train/0001.jpg', 0, 425.3, 641.7, 681.9, 861.7], ['train/0001.jpg', 7, 92.4, 231.6, 601.7, 654.8000000000001], ['train/0001.jpg', 0, 622.4, 695.1999999999999, 686.5, 780.7], ['train/0002.jpg', 3, 267.9, 899.5, 165.2, 678.2]]


In [8]:
print(new_pred[:10])

[['train/0001.jpg', '7', '0.8273342', 0.4756622314453125, 155.73626708984375, 450.24151611328125, 647.1145629882812], ['train/0001.jpg', '4', '0.68532103', 719.5620727539062, 986.5067749023438, 295.9759521484375, 569.048828125], ['train/0001.jpg', '7', '0.45605806', 92.7458724975586, 235.04776000976562, 597.154296875, 659.9893798828125], ['train/0001.jpg', '9', '0.37374008', 27.6329345703125, 775.2059326171875, 442.7140197753906, 741.82861328125], ['train/0001.jpg', '4', '0.35860425', 40.368133544921875, 779.786376953125, 439.53350830078125, 741.7778930664062], ['train/0001.jpg', '4', '0.29569444', 357.2955322265625, 572.0662231445312, 663.8292236328125, 775.4056396484375], ['train/0001.jpg', '5', '0.2738941', 357.2955322265625, 572.0662231445312, 663.8292236328125, 775.4056396484375], ['train/0001.jpg', '7', '0.19315323', 46.160850524902344, 236.86734008789062, 450.83441162109375, 651.1080932617188], ['train/0001.jpg', '9', '0.19230662', 46.160850524902344, 236.86734008789062, 450.834

In [6]:
'''
calculate mAP
'''


mean_ap, average_precisions = mean_average_precision_for_boxes(gt, new_pred, iou_threshold=0.5)

print(mean_ap)

Number of files in annotations: 4883
Number of files in predictions: 4625
Unique classes: 10
Detections length: 4625
Annotations length: 4883
0                              | 0.000000 |    3966
1                              | 0.646945 |    6352
2                              | 0.713611 |     897
3                              | 0.640260 |     936
4                              | 0.534331 |     982
5                              | 0.618766 |    2943
6                              | 0.675804 |    1263
7                              | 0.804650 |    5178
8                              | 0.466301 |     159
9                              | 0.641704 |     468
mAP: 0.574237
0.5742372755188985
